<a href="https://colab.research.google.com/github/Beulah-Karrolla/pyTorch_files/blob/Lab0/Lab0_ResNet18_SVHN_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import PyTorch and Torchvision
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import time

In [ ]:
# Download training data from open datasets.
training_data = datasets.SVHN(
    root="data",
    split='train',
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.SVHN(
    root="data",
    split='test',
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/182040794 [00:00<?, ?it/s]

  0%|          | 0/64275384 [00:00<?, ?it/s]

In [ ]:
print(training_data)

Dataset SVHN
    Number of datapoints: 73257
    Root location: data
    Split: train
    StandardTransform
Transform: ToTensor()


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
# my :: n-batch size, c-channel(1:grey scale image), h:height, w:width
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
from torchvision import models

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
net = models.resnet18(pretrained=True)
#net = net.cuda() if device else net
model = net.to(device)
print(model)

Using cpu device


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
print(model.parameters())

<generator object Module.parameters at 0x7f896a14b550>


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    t_start = time.time()
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad() #sgd : till gradient is zero
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    print(f"Time taken:{time.time()-t_start:>3f}")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 12.697556  [    0/73257]
loss: 4.209089  [ 6400/73257]
loss: 3.351606  [12800/73257]
loss: 2.344321  [19200/73257]
loss: 2.089396  [25600/73257]
loss: 1.859959  [32000/73257]
loss: 1.610016  [38400/73257]
loss: 1.877005  [44800/73257]
loss: 1.337773  [51200/73257]
loss: 1.454438  [57600/73257]
loss: 1.206450  [64000/73257]
loss: 1.534759  [70400/73257]
Time taken:774.329844
Test Error: 
 Accuracy: 61.4%, Avg loss: 1.194850 

Epoch 2
-------------------------------
loss: 1.111241  [    0/73257]
loss: 0.940839  [ 6400/73257]
loss: 1.347930  [12800/73257]
loss: 0.974181  [19200/73257]
loss: 0.881291  [25600/73257]
loss: 0.959664  [32000/73257]
loss: 0.772784  [38400/73257]
loss: 1.238717  [44800/73257]
loss: 0.673768  [51200/73257]
loss: 0.812575  [57600/73257]
loss: 0.834697  [64000/73257]
loss: 0.966801  [70400/73257]
Time taken:775.353001
Test Error: 
 Accuracy: 73.8%, Avg loss: 0.828606 

Epoch 3
-------------------------------
loss: 0.663